## Scraping Data Scientist jobs from Seek.com.au using Beautiful Soup

In [1]:
# Importing libraries
import re
import pandas as pd
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
from bs4 import BeautifulSoup
import json
from time import sleep

In [2]:
# Choosing two salary ranges for each job, which are the equivalent of Low and High
salary_ranges = ['50000-80000', '80000-120000', '120000-150000']

# Dictionary to store all the web links
job_urls = {}

for salary_range in salary_ranges:
    job_urls[salary_range]=[]
    for page in range(1,40):
        # iterate through the first 40 pages of jobs
        r = requests.get("https://www.seek.com.au/data-scientist-jobs/in-All-Sydney-NSW?page={0}&salaryrange={1}&salarytype=annual".format(str(page),salary_range))

        # turn into a BeautifulSoup object
        soup = BeautifulSoup(r.text, 'lxml')

        # find urls on page
        url = soup.find_all(attrs={'data-automation':"jobTitle"})
        /
        if url != []: 
            job_urls[salary_range].extend(url)

In [3]:
df_1 = pd.DataFrame()
df_1['Link'] = job_urls['50000-80000']
df_1['Salary Range'] = '50000-80000'

In [4]:
df_1.head()

,Link,Salary Range
0,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000
1,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000
2,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000
3,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000
4,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000


In [5]:
df_2 = pd.DataFrame()
df_2['Link'] = job_urls['80000-120000']
df_2['Salary Range'] = '80000-120000'

In [6]:
df_3 = pd.DataFrame()
df_3['Link'] = job_urls['120000-150000']
df_3['Salary Range'] = '120000-150000'

In [7]:
url_pd = df_1.append(df_2).append(df_3)

In [8]:
url_pd.tail()

,Link,Salary Range
126,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000
127,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000
128,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000
129,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000
130,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000


In [9]:
url_pd['Link_1'] = ['https://www.seek.com.au' + re.search('(\/job\/[0-9]{8})',i.attrs['href']).group(0) for i in url_pd['Link']]

In [10]:
url_pd.head()

,Link,Salary Range,Link_1
0,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40077526
1,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40089444
2,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40278562
3,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40270045
4,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40270044


In [11]:
url_pd['Title'] = [i.text for i in url_pd['Link']]

In [12]:
url_pd = url_pd.drop('Link',axis='columns')

In [13]:
url_pd.head()

,Salary Range,Link_1,Title
0,50000-80000,https://www.seek.com.au/job/40077526,Full Stack Software Developer
1,50000-80000,https://www.seek.com.au/job/40089444,Experienced Actuarial and Analytics Consultant...
2,50000-80000,https://www.seek.com.au/job/40278562,Research Fellow - Data Scientist
3,50000-80000,https://www.seek.com.au/job/40270045,Data Scientist \ Engineer
4,50000-80000,https://www.seek.com.au/job/40270044,Data Scientist \ Engineer


In [14]:
columns = ['Salary Range', 'Link', "Title"]
url_pd.columns=columns

In [15]:
url_pd.head()

,Salary Range,Link,Title
0,50000-80000,https://www.seek.com.au/job/40077526,Full Stack Software Developer
1,50000-80000,https://www.seek.com.au/job/40089444,Experienced Actuarial and Analytics Consultant...
2,50000-80000,https://www.seek.com.au/job/40278562,Research Fellow - Data Scientist
3,50000-80000,https://www.seek.com.au/job/40270045,Data Scientist \ Engineer
4,50000-80000,https://www.seek.com.au/job/40270044,Data Scientist \ Engineer


In [16]:
#for stemming
import re

def replace(string, substitutions):

    substrings = sorted(substitutions, key=len, reverse=True)
    regex = re.compile('|'.join(map(re.escape, substrings)))
    return regex.sub(lambda match: substitutions[match.group(0)], string)

In [17]:
job_listing_date=[]
job_expiry_date=[]
job_title = []
job_teaser=[]
job_advertiser=[]
job_area=[]
job_worktype=[]
job_classification=[]
job_salary=[]
job_salary_type=[]
job_description_details=[]
job_description_strong = []
job_location =[]

for index, job in enumerate(url_pd['Link']):
    # iterate through all the urls
    try:
        r = requests.get(job)
    except:
        time.sleep(5)
    
    # turn into a BeautifulSoup object
    soup = BeautifulSoup(r.text, 'lxml')
    
    # find job descriptions
    jd_data = soup.find_all(attrs={'data-automation':'server-state'})[0]
    jd_raw= jd_data.text[3:].split('\n  ')[1][25:][:-1]
    jd_dict = json.loads(jd_raw)
    jd_dashboard = jd_dict['jobdetails']['result']
    
    if jd_dashboard['listingDate']:
        job_listing_date.append(jd_dashboard['listingDate'])
    else:
        job_listing_date.append('N/A')
        
    if jd_dashboard['expiryDate']:
        job_expiry_date.append(jd_dashboard['expiryDate'])
    else:
        job_expiry_date.append('N/A')
        
    if jd_dashboard['title']:
        job_title.append(jd_dashboard['title'])
    else:
        job_title.append('N/A')
        
    if jd_dashboard['teaser']:
        job_teaser.append(jd_dashboard['teaser'])
    else:
        job_teaser.append('N/A')
        
    if jd_dashboard['advertiser']['description']:
        job_advertiser.append(jd_dashboard['advertiser']['description'])
    else:
        job_advertiser.append('N/A')
        
    if jd_dashboard['locationHierarchy']['area']:
        job_area.append(jd_dashboard['locationHierarchy']['area'])
    else:
        job_area.append('N/A')
        
    if jd_dashboard['workType']:
        job_worktype.append(jd_dashboard['workType'])
    else:
        job_worktype.append('N/A')
        
    if jd_dashboard['classification']['description']:
        job_classification.append(jd_dashboard['classification']['description'])
    else:
        job_classification.append('N/A')
        
    if jd_dashboard['salary']:
        job_salary.append(jd_dashboard['salary'])
    else:
        job_salary.append('N/A')
        
    if jd_dashboard['salaryType']:
        job_salary_type.append(jd_dashboard['salaryType'])
    else:
        job_salary_type.append('N/A')
        
    if jd_dashboard['locationHierarchy']['city']:
        job_location.append(jd_dashboard['locationHierarchy']['city'])
    else:
        job_location.append('N/A')
        
    
    jd_detail = soup.find(attrs={'data-automation':'mobileTemplate'})
    job_description_details.append(jd_detail)
    

In [18]:
url_pd['Listing Date'] = job_listing_date
url_pd['Expiry Date'] =job_expiry_date
url_pd['Job Title']=job_title
url_pd['Job Teaser']=job_teaser
url_pd['Advertiser'] = job_advertiser
url_pd['Area'] = job_area
url_pd['Work Type'] = job_worktype
url_pd['Classification'] = job_classification
url_pd['Salary'] = job_salary
url_pd['Salary Type'] = job_salary_type
url_pd['JD'] = job_description_details
url_pd['Location'] =job_location

In [19]:
# Check the type for 'JD' column and start cleaning
type(job_description_details)

list

In [20]:
job_description_clean=[]
job_description_strong = []
for i in range(0,len(url_pd['JD'])):
    try:
        strong_word = job_description_details[i].find_all('strong')
        strong_word_list = [u.get_text(strip=True) for u in strong_word if strong_word != False]
        job_description_strong.append(strong_word_list)

        string = job_description_details[i].get_text()
        repla = {u'\xa0':'  ',u'\xe2\x80\x9d':'  ', u'\n':'  '}
        job_description_clean.append(replace(string,repla))
    except:
        job_description_strong.append('N/A')
        job_description_clean.append('N/A')

In [21]:
url_pd['Job Description'] = job_description_clean
url_pd['Strong Words'] = job_description_strong 

In [22]:
data_scientist_jobs = url_pd.copy()

In [23]:
data_scientist_jobs.to_csv('data_scientist_jobs.csv')

In [24]:
data_scientist_jobs.tail(10)

,Salary Range,Link,Title,Listing Date,Expiry Date,Job Title,Job Teaser,Advertiser,Area,Work Type,Classification,Salary,Salary Type,JD,Location,Job Description,Strong Words
121,120000-150000,https://www.seek.com.au/job/40121990,Senior Systems Engineer (Level 3),2019-10-09T00:46:38.000Z,2019-11-08T12:59:59.000Z,Senior Systems Engineer (Level 3),Looking for a Senior Systems Engineer to take ...,Talentpool Recruitment Group,"CBD, Inner West & Eastern Suburbs",Contract/Temp,Information & Communication Technology,N/A,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,The OrganisationA global algo-trading firm at ...,"[The Organisation, The Role, About You, Key Sk..."
122,120000-150000,https://www.seek.com.au/job/40139637,Marketing Manager,2019-10-10T23:12:46.000Z,2019-11-10T12:59:59.000Z,Marketing Manager,We are currently looking for a Digital Marketi...,Stockland,"CBD, Inner West & Eastern Suburbs",Contract/Temp,Marketing & Communications,N/A,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,We are currently looking for a Digital Marketi...,"[Your main duties will involve, but will not b..."
123,120000-150000,https://www.seek.com.au/job/40280331,Environmental Engineer/Scientist,2019-10-30T05:51:39.000Z,2019-11-29T12:59:59.000Z,Environmental Engineer/Scientist,Progressive waste management business. Support...,Constructive Recruitment Pty Ltd,Southern Suburbs & Sutherland Shire,Full Time,Engineering,Negotiable depending on experience,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,What's on offer: Progressive waste manageme...,"[What's on offer:, The Company:, The Role:, Ke..."
124,120000-150000,https://www.seek.com.au/job/40096686,Senior DevOps Engineer (Systems Engineering),2019-10-04T04:26:21.000Z,2019-11-03T04:26:21.000Z,Senior DevOps Engineer (Systems Engineering),We're looking for a talented Senior DevOps Eng...,Deputy.com,"CBD, Inner West & Eastern Suburbs",Full Time,Information & Communication Technology,N/A,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,Why join? Australian Hyper-Growth Startup - He...,"[Why join?, Your Mission, What you will do., H..."
125,120000-150000,https://www.seek.com.au/job/40053039,Software Engineer,2019-09-30T01:34:53.000Z,2019-10-30T12:59:59.000Z,Software Engineer,"You spend most of your life at work, why join ...",The Onset,"CBD, Inner West & Eastern Suburbs",Full Time,Information & Communication Technology,"$100,000 - $130,000 + super",AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,You spend most of your life at work and it’s w...,"[what is it?, What do they want from you?, ]"
126,120000-150000,https://www.seek.com.au/job/40250508,Head - Computational Biology,2019-10-25T05:53:00.000Z,2019-11-24T12:59:59.000Z,Head - Computational Biology,We're looking for a Head - Computational Biolo...,Garvan Institute of Medical Research,"CBD, Inner West & Eastern Suburbs",Full Time,Science & Technology,Salary Packaging Benefits,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,The Garvan Institute of Medical Research (Garv...,"[The Opportunity, About You, How to Apply, Clo..."
127,120000-150000,https://www.seek.com.au/job/40248180,Account Manager NSW - Molecular Diagnostics,2019-10-25T03:59:19.000Z,2019-11-24T03:59:19.000Z,Account Manager NSW - Molecular Diagnostics,Promote the best in class product portfolio th...,QIAGEN Pty Ltd,"CBD, Inner West & Eastern Suburbs",Full Time,Science & Technology,N/A,AnnualCommission,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,About the business QIAGEN is the leading glob...,"[About the business, About the role, Account M..."
128,120000-150000,https://www.seek.com.au/job/40187927,Senior UI/UX Designer,2019-10-17T04:40:53.000Z,2019-11-16T04:40:52.000Z,Senior UI/UX Designer,Solar Analytics is looking for a senior UX des...,Solar Analytics,"CBD, Inner West & Eastern Suburbs",Full Time,Design & Architect